# Projeto 2 - Ciência dos Dados

Nome: Roberto Schnorrenberg

Nome: Gabriel Benatar

Para o projeto dois, a captura e análise dos dados será realizada sobre o produto skol beats.

# Proposta:

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa selecione algumas mensagens disponíveis no Twitter que mencionam esse produto e classifique-as como "relevante" ou "irrelevante".
Com isso, essa empresa deseja que mensagens relevantes, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti- spam de e-mails, por exemplo. Esse classificador permite calcular qual a probabilidade de uma mensagem ser relevante dada as palavras em seu conteúdo.
Para realizar o MVP (minimum viable product) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.


___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [1]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [2]:
import tweepy
import math
import os.path
%matplotlib inline
import pandas as pd
import numpy as np
import math as math
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import json
from random import shuffle
from IPython.display import display
import numpy as np

___
## Autenticando no  Twitter

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***

In [3]:
import re 
def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

In [4]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [5]:
#Produto escolhido:
produto = 'skol beats'

#Quantidade mínima de mensagens capturadas:
n = 950
#Quantidade mínima de mensagens para a base de treinamento:
t = 500

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'


Capturando os dados do twitter:

In [6]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []

for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    msgs.append(msg.full_text.lower())
    i += 1
    if i > n:
        break

    
#tirando os rts das mensagens selecionadas.

for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    if msg.full_text.lower()[0] != 'r' and msg.full_text.lower()[1] != 't':
        msgs.append(msg.full_text.lower())
        i += 1
        if i > n:
            break

#Tentando tirar as iguais  

msgs=list(set(msgs))

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

TweepError: Failed to send request: HTTPSConnectionPool(host='api.twitter.com', port=443): Max retries exceeded with url: /1.1/search/tweets.json?q=skol+beats&lang=pt&tweet_mode=extended (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000001ECD6255E80>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

Salvando os dados em uma planilha Excel:

In [ ]:

#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

In [7]:
#Separando as tabelas do excel.
twitts_tre = pd.read_excel('skol_beats.xlsx')
twitts_tes = pd.read_excel(io='skol_beats.xlsx', sheet_name='Teste')

In [8]:
#limpando o treinamento e deixando tudo minuscula.
twitts_tre['Treinamento']=twitts_tre['Treinamento'].apply(cleanup)
twitts_tre['Treinamento']=twitts_tre['Treinamento'].str.lower()

#limpando o teste e deixando tudo minuscula.
twitts_tes['Teste']=twitts_tes['Teste'].apply(cleanup)
twitts_tes['Teste']=twitts_tes['Teste'].str.lower()


#Separando nos grupos de relevancia do treinamento.
relevante = twitts_tre[twitts_tre['RELEVANCIA']==1]
irrelevante = twitts_tre[twitts_tre['RELEVANCIA']==0]

#Separando nos grupos de relevanica do teste.
rel_tes = twitts_tes[twitts_tes['RELEVANCIA']==1]
irr_tes = twitts_tes[twitts_tes['RELEVANCIA']==0]

In [9]:
twitts_tes.RELEVANCIA = twitts_tes.RELEVANCIA.astype('category')
twitts_tes.RELEVANCIA.cat.categories = ('Irrelevante', 'Relevante')
twitts_tes.head()

,Teste,RELEVANCIA
0,skol beats pq tão boa,Irrelevante
1,mds skol beats é muito bom não acredito que t...,Irrelevante
2,tomei uma latinha de skol beats e me deu um so...,Irrelevante
3,o bairro seus pode até ser da hora mas aqui d...,Relevante
4,foi uma péssima ideia ir bebendo skol beats en...,Irrelevante


In [10]:
twitts_tes.RELEVANCIA.value_counts(True)*100

Irrelevante    70.0
Relevante      30.0
Name: RELEVANCIA, dtype: float64

In [11]:
#adicionando as palavras relevantes e irrelevantes a uma lista.
palavras_rel = relevante['Treinamento'].str.split()
palavras_irr = irrelevante['Treinamento'].str.split()
pal_tes_rel = rel_tes['Teste'].str.split()
pal_tes_irr = irr_tes['Teste'].str.split()

#lista de palavras relevantes.
palavras_rel_lista=[]
for i in palavras_rel:
    for e in i:
        palavras_rel_lista.append(e)
        
#lista de palavras irrelevantes.
palavras_irr_lista=[]
for x in palavras_irr:
    for y in x:
        palavras_irr_lista.append(y)

#lista de palavras relevantes do teste
pal_rel_tes=[]
for i in pal_tes_rel:
    for e in i:
        pal_rel_tes.append(e)

#lista de palavras irrelevantes do teste.
pal_irr_tes=[]
for ty in pal_tes_irr:
    for tu in ty:
        pal_irr_tes.append(tu)
        
#Total de palavras diferntes na lista.
total_pal=[]
for i in palavras_rel_lista:
    if i not in total_pal:
        total_pal.append(i)
for i in palavras_irr_lista:
    if i not in total_pal:
        total_pal.append(i)

In [12]:
#limpando as palavras irrelevantes das listas do teste.
for pal in pal_rel_tes:
        if '@' in pal:
            pal_rel_tes.remove(pal)
        if 'http' in pal:
            pal_rel_tes.remove(pal)
        if 'kkk' in pal:
            pal_rel_tes.remove(pal)
        if '/' in pal:
            pal_rel_tes.remove(pal)
        if 'ksksks' in pal:
            pal_rel_tes.remove(pal)

for pal in pal_irr_tes:
        if '@' in pal:
            pal_irr_tes.remove(pal)
        if 'http' in pal:
            pal_irr_tes.remove(pal)
        if 'kkk' in pal:
            pal_irr_tes.remove(pal)
        if '/' in pal:
            pal_irr_tes.remove(pal)
        if 'ksksks' in pal:
            pal_irr_tes.remove(pal)

In [13]:
quant_pal_relt = len(pal_rel_tes)
quant_pal_irrt = len(pal_irr_tes)

print('Do total de 6204 palavras em análise no teste: \n')
print('A lista das palavras relevantes possui {0} palavras.'.format(quant_pal_relt))
print('A lista das palavras irrelevantes possui {0} palavras.'.format(quant_pal_irrt))

Do total de 6204 palavras em análise no teste: 

A lista das palavras relevantes possui 2003 palavras.
A lista das palavras irrelevantes possui 4273 palavras.


In [14]:
#limpando as palavras irrelevantes das listas.
for pal in palavras_rel_lista:
        if '@' in pal:
            palavras_rel_lista.remove(pal)
        if 'http' in pal:
            palavras_rel_lista.remove(pal)
        if 'kkk' in pal:
            palavras_rel_lista.remove(pal)
        if '/' in pal:
            palavras_rel_lista.remove(pal)
        if 'ksksks' in pal:
            palavras_rel_lista.remove(pal)

for pal in palavras_irr_lista:
        if '@' in pal:
            palavras_irr_lista.remove(pal)
        if 'http' in pal:
            palavras_irr_lista.remove(pal)
        if 'kkk' in pal:
            palavras_irr_lista.remove(pal)
        if '/' in pal:
            palavras_irr_lista.remove(pal)
        if 'ksksks' in pal:
            palavras_irr_lista.remove(pal)

quant_pal_rel = len(palavras_rel_lista)
quant_pal_irr = len(palavras_irr_lista)

print('Do total de 7232 palavras em análise no treinamento: \n')
print('A lista das palavras relevantes possui {0} palavras.'.format(quant_pal_rel))
print('A lista das palavras irrelevantes possui {0} palavras.'.format(quant_pal_irr))

Do total de 7232 palavras em análise no treinamento: 

A lista das palavras relevantes possui 2276 palavras.
A lista das palavras irrelevantes possui 4956 palavras.


In [15]:
#frequencia relativa das palavras relevantes.
serie_rel = pd.Series(palavras_rel_lista)
tab_serie_rel = serie_rel.value_counts(True)

#freuencia relativa das plavras irrelevantes.
serie_irr = pd.Series(palavras_irr_lista)
tab_serie_irr = serie_irr.value_counts(True)

#frequencia relativa do total de palavras.
serie_total = pd.Series(total_pal)
tab_serie_total = serie_total.value_counts(True)

In [16]:
total_pal = twitts_tre['Treinamento'].str.split()
total_pal_tes = twitts_tes['Teste'].str.split()

#dataframe para as frases relevant6es e irrelevantes
df_pr = pd.DataFrame(np.array(palavras_rel), columns=['Frases relevantes'])
df_pi = pd.DataFrame(np.array(palavras_irr), columns=['Frases irrelevantes'])
df_prt = pd.DataFrame(np.array(pal_tes_rel), columns = ['Frases relevantes do teste'])
df_pit = pd.DataFrame(np.array(pal_tes_irr), columns = ['Frases irrelevantes do teste'])

#lista das frases dos twitts relevantes.
lista_frase_rel=[]
for i in df_pr['Frases relevantes']:
    lista_frase_rel.append(i)
#lista_frase_rel

#lista das frases dos twitts irrelevantes.
lista_frase_irr=[]
for i in df_pi["Frases irrelevantes"]:
    lista_frase_irr.append(i)

#a frequncia que cada palavra possui em cada lista, lista relevante:
freq_rel={}
for palavra in palavras_rel_lista:
    if palavra not in freq_rel:
        freq_rel[palavra]=1
    else:
        freq_rel[palavra]+=1

#A frequencia das palavras na lista irrelevante.
freq_irr={}
for palavra in palavras_irr_lista:
    if palavra not in freq_irr:
        freq_irr[palavra]=1
    else:
        freq_irr[palavra]+=1

In [17]:
#Fazendo uma lista para as frases do teste.
ttr = []
for listas in df_prt["Frases relevantes do teste"]:
    ttr.append(listas)

tti = []
for listas in df_pit["Frases irrelevantes do teste"]:
    tti.append(listas) 

#fazendo as frequencia da presença das palavras agora para o teste.
frt={}
for pal in pal_rel_tes:
    if pal not in frt:
        frt[pal]=1
    else:
        frt[pal]+=1
fit={}
for pal in pal_irr_tes:
    if pal not in fit:
        fit[pal]=1
    else:
        fit[pal]+=1

In [18]:
#A probabilidade de cada palavra aparecer na lista relevante.
prob_rel={}
for i in freq_rel:
    p = (freq_rel[i])/(len(palavras_rel_lista))
    prob_rel[i] = p
#A probabilidade de cada palavra aparecer na lista irrelevante.    
prob_irr={}
for i in freq_irr:
    p = (freq_irr[i])/(len(palavras_irr_lista))
    prob_irr[i] = p

#A probabilidade de cada palavra aparecer na lista relevante do teste.
prob_rel_tes={}
for t in frt:
    po = (frt[t])/(len(pal_rel_tes))
    prob_rel_tes[i] = po
    
#A probabilidade de cada palavra aparecer na lista irrelevante do teste.    
prob_irr_tes={}
for t in fit:
    po = (fit[t])/(len(pal_irr_tes))
    prob_irr_tes[i] = po

In [19]:
#vendo se o twitt é relevante.
ptr=[]
p=1
for i in ptr:
    for t in prob_rel:
        if t in i:
            p=p*prob_rel[t]
        ptr.append(p)
        p=1
        
#vendo se o twitt é irrelevante.
pti=[]
po=1
for x in pti:
    for y in prob_irr:
        if y in x:
            po=po*prob_irr[y]
        pti.append(po)
        po=1

#vendo se o twitt é relevante.
ptrt=[]
p2=1
for i in ptrt:
    for t in prob_rel_tes:
        if t in i:
            p=p*prob_rel_tes[t]
        ptrt.append(p2)
        p2=1
#vendo se o twitt é irrelevante.
ptit=[]
po2=1
for x in ptit:
    for y in prob_irr_tes:
        if y in x:
            po=po*prob_irr_tes[y]
        ptit.append(po2)
        po2=1

In [20]:
#probabilidade do treinamento.
probabilidade_relevante = len(palavras_rel)/len(twitts_tre)
probabilidade_irrelevante = len(palavras_irr)/len(twitts_tre)
#probabilidade do teste.
po_re = len(pal_rel_tes)/len(twitts_tes)
po_ir = len(pal_irr_tes)/len(twitts_tes)

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

# Treinamento

In [21]:
lap_rel=[]
lap_irr=[]

for i in lista_frase_rel:
    ob=1
    for palavra in i:
        if palavra in freq_rel:
            ob=ob*(freq_rel[palavra]+1/(len(palavras_rel_lista)+len(total_pal)))
        else:
            ob=ob*(1/len(palavras_rel_lista)+len(total_pal))
    prob1= ob*probabilidade_relevante
    lap_rel.append(prob1)
    ob=1
    po=1
    for s in i:
        if s in freq_irr:
            po=po*(freq_irr[s]+1/(len(palavras_irr_lista)+len(total_pal)))
        else:
            po=po*(1/len(palavras_rel_lista)+len(total_pal))
    prob2=probabilidade_irrelevante*po
    lap_irr.append(prob2)
    po=1

In [22]:
critica_relevante_construtiva=[]
critica_relevante_destrutiva=[]

for i in range(0,len(lap_rel)):
    if lap_rel[i]>lap_irr[i]:
        critica_relevante_construtiva.append('ta_safe')
    else:
        critica_relevante_destrutiva.append('ta_safe')
print('Dos twitts análisados como relevantes: {0} foram marcados como relevantes e {1} como irrelevantes.'.format(len(critica_relevante_construtiva),len(critica_relevante_destrutiva)))

Dos twitts análisados como relevantes: 1 foram marcados como relevantes e 155 como irrelevantes.


In [23]:
lap_rel=[]
lap_irr=[]

for i in lista_frase_rel:
    ob=1
    for palavra in i:
        if palavra in freq_rel:
            ob=ob*(freq_rel[palavra]+1/(len(palavras_rel_lista)+len(total_pal)))
        else:
            ob=ob*(1/len(palavras_rel_lista)+len(total_pal))
    prob1= ob*probabilidade_relevante
    lap_rel.append(prob1)
    ob=1
    po=1
    for s in i:
        if s in freq_irr:
            po=po*(freq_irr[s]+1/(len(palavras_irr_lista)+len(total_pal)))
        else:
            po=po*(1/len(palavras_rel_lista)+len(total_pal))
    prob2=po*probabilidade_irrelevante
    lap_irr.append(prob2)
    po=1

In [24]:
critica_irrelevante_construtiva=[]
critica_irrelevante_destrutiva=[]

for i in range(0,len(lap_irr)):
    if lap_irr[i]>lap_rel[i]:
        critica_irrelevante_construtiva.append('ta_safe')
    else:
        critica_irrelevante_destrutiva.append('ta_safe')
print('Dos twitts análisados como irrelevantes: {0} foram marcados como relevantes e {1} como irrelevantes.'.format(len(critica_irrelevante_construtiva),len(critica_irrelevante_destrutiva)))

Dos twitts análisados como irrelevantes: 155 foram marcados como relevantes e 1 como irrelevantes.


# Teste

In [25]:
lap_rel_tes=[]
lap_irr_tes=[]

for i in ttr:
    ob=1
    for palavra in i:
        if palavra in frt:
            ob=ob*(frt[palavra]+1/(len(pal_rel_tes)+len(total_pal_tes)))
        else:
            ob=ob*(1/len(pal_rel_tes)+len(total_pal))
    prob1= ob*po_re
    lap_rel_tes.append(prob1)
    ob=1
    po=1
    for s in i:
        if s in fit:
            po=po*(fit[s]+1/(len(pal_irr_tes)+len(total_pal_tes)))
        else:
            po=po*(1/len(pal_rel_tes)+len(total_pal_tes))
    prob2=po_ir*po
    lap_irr_tes.append(prob2)
    po=1

In [26]:
verdade_construtiva=[]
falso_destrutivo=[]

for e in range(0,len(lap_rel_tes)):
    if lap_rel_tes[e] > lap_irr_tes[e]:
        verdade_construtiva.append('ta_safe')

for i in range(0,len(lap_irr_tes)):
    if lap_irr_tes[i]>lap_rel_tes[i]:
        falso_destrutivo.append('ta_safe')

print('Dos twitts análisados como irrelevantes: {0} foram marcados como relevantes e {1} como irrelevantes.'.format(len(verdade_construtiva),len(falso_destrutivo)))

Dos twitts análisados como irrelevantes: 0 foram marcados como relevantes e 132 como irrelevantes.


In [27]:
lap_rel_tes=[]
lap_irr_tes=[]

for i in tti:
    ob=1
    for palavra in i:
        if palavra in frt:
            ob=ob*(frt[palavra]+1/(len(pal_rel_tes)+len(total_pal_tes)))
        else:
            ob=ob*(1/len(pal_rel_tes)+len(total_pal))
    prob1= ob*po_re
    lap_rel_tes.append(prob1)
    ob=1
    po=1
    for s in i:
        if s in fit:
            po=po*(fit[s]+1/(len(pal_irr_tes)+len(total_pal_tes)))
        else:
            po=po*(1/len(pal_rel_tes)+len(total_pal_tes))
    prob2=po_ir*po
    lap_irr_tes.append(prob2)
    po=1

In [28]:
verdade_destrutiva=[]
falso_construtivo=[]

for e in range(0,len(lap_rel_tes)):
    if lap_rel_tes[e] > lap_irr_tes[e]:
        verdade_destrutiva.append('ta_safe')

for i in range(0,len(lap_irr_tes)):
    if lap_irr_tes[i]>lap_rel_tes[i]:
        falso_construtivo.append('ta_safe')

print('Dos twitts análisados como irrelevantes: {0} foram marcados como relevantes e {1} como irrelevantes.'.format(len(verdade_destrutiva),len(falso_construtivo)))

Dos twitts análisados como irrelevantes: 250 foram marcados como relevantes e 58 como irrelevantes.


___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [29]:
linha = ['Relevante','Irrelevante']
coluna = ['verdade','falso']

ved_cons=len(verdade_construtiva)/len(twitts_tes)
fal_des=len(falso_destrutivo)/len(twitts_tes)
ved_des=len(verdade_destrutiva)/len(twitts_tes)
fal_cons=len(falso_construtivo)/len(twitts_tes)

t=[[ved_cons,fal_des],[ved_des,fal_cons]]

ct = pd.DataFrame(np.array(t), index=linha, columns = coluna)
ct

,verdade,falso
Relevante,0.000000,0.300000
Irrelevante,0.568182,0.131818


___
### Concluindo

Em nosso projeto usamos o classificador de Naive Bayes, ele foi baseado no teorema de Bayes, e é muito usado em machine learning, classificação de textos, filtragens etc. Em nosso projeto escolhemos skol beats com tema dos tweets a serem analisados. Nele usamos o classificador de Bayes para filtrar os tweets e separá los entre “relevante” e “irrelevante” , para que a empresa possa analisar melhor os comentários para que a empresa possa entender melhor seu público e o que ele deseja.
O nosso classificador apresenta falhar, porém há propostas de melhorias sobre ele. Ele teve uma eficácia baixa de acordo com a classificação nas base de treinamento e teste, pois após a realização do laplace para as duas bases fica explícito que a base de treinamento e teste não correspondem a quantidades semelhantes de twitts relevantes e irrelevantes.

As mensagens de dupla negação e sarcasmo foram tratadas normalmente, como “irrelevantes” ou “relevantes”, mesmo sendo sarcástica. Isso pode ter sido um dos fatores que possa ter diminuído a eficácia do classificador.
Nestes casos de classificação não seria possível alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets, pois a base de treinamento é onde será ensiado para o computador quais são as palavras que devem ser relevantes ou irrelevantes, então se for aplicado na base de treinamento automaticamente, não haverá a base de entendimento, ou seja, se você não ensinar um padrão ao computador e colocar direto a base de dados ele não entenderá.

Nestes casos de classificação não seria possível alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets, pois a base de treinamento é onde será ensiado para o computador quais são as palavras que devem ser relevantes ou irrelevantes, então se for aplicado na base de treinamento automaticamente, não haverá a base de entendimento, ou seja, se você não ensinar um padrão ao computador e colocar direto a base de dados ele não entenderá.
Para que numa próxima iteração tivéssemos mais sucesso poderíamos consertar esses pequenos erros e também fazer talvez uma base de treinamento com mais tweets fazendo com que abranja uma maior quantidade de palavras podendo aumentar na sua eficácia.


## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**